## Inversion of Control
Spring framework demonstrates *inversion of control* by moving the responsibility of providing dependencies away from the user to the framework through *dependency injection*. The terms *dependency injection* and *inversion of control* are explained [here](https://martinfowler.com/articles/injection.html).

## IoC Container
`BeanFactory` interface forms the basis of Spring IOC container. `BeanFactory` provides few basic functionalities to manage beans, the most important being the ability to get a bean using its name. A bean is an object that is instantiated, assembled, and otherwise managed by a Spring IoC container.

In [ ]:
public interface BeanFactory {
    // Will have to explicitly cast to the required type
    Object getBean(String name) throws BeansException;
    // No casting required!
    <T> T getBean(String name, Class<T> requiredType) throws BeansException;
    
    // Many other methods
}

`BeanFactory` creates beans from bean definitions - defined using XML or in Java itself. `BeanDefinition` is the blueprint of a bean instance. Some of the information contained within the `BeanDefinition` is highlighted using the source of the interface below:

In [ ]:
public interface BeanDefinition extends AttributeAccessor, BeanMetadataElement {
    // Whether to initialize the bean lazily
    boolean isLazyInit();
    
    // What is the scope of the bean? Singleton, Prototype, etc?
    @Nullable
    String getScope();
    
    // Is the bean going to be the primary autowire candidate?
    boolean isPrimary();
    
    // Provides Type information
    ResolvableType getResolvableType();
    
    @Nullable
    String getFactoryMethodName();
    
    @Nullable
    String getInitMethodName();
    
    @Nullable
    String getDestroyMethodName();
    
    // Other methods
}

`BeanDefinitions` are registered in `BeanDefinitionRegistry`:

In [ ]:
public interface BeanDefinitionRegistry extends AliasRegistry {

    void registerBeanDefinition(String beanName, BeanDefinition beanDefinition)
            throws BeanDefinitionStoreException;

    void removeBeanDefinition(String beanName) throws NoSuchBeanDefinitionException;

    BeanDefinition getBeanDefinition(String beanName) throws NoSuchBeanDefinitionException;

    boolean containsBeanDefinition(String beanName);
    
    // Other methods
}

As an example, if we have a bean defined using configuration class below:

In [ ]:
@Configuration
public class ApplicationConfiguration {
    @Bean
    public CurrencyService getCurrencyService() {
        return new CurrencyService();
    }
}

<style>
    table {
        display: inline-block
    }
</style>
The `BeanDefinition` for the `CurrencyService` bean above, looks like:
- Type: `CurrencyService`
- Factory bean: `ApplicationConfiguration`
- Factory method: `getCurrencyService()`

The two interfaces `BeanFactory` and `BeanRegistry` come together in `DefaultListableBeanFactory`. This class stores all the `BeanDefinitions` in a concurrent hashmap with key pointing to the bean name and value pointing to the bean definition. `DefaultListableBeanFactory` forms the heart of the Spring framework since it manages the bean definitions as well as the bean instances.

### ApplicationContext
`ApplicationContext` interface extends `BeanFactory` and adds some additional features. Simple comparison:  
|                                    | ApplicationContext | BeanFactory |
|------------------------------------|--------------------|-------------|
| Annotation support                 | Yes                | No          |
| `BeanPostProcessor` Registration   | Auto               | Manual      |
| Internationalization               | Yes                | No          |
| Enterprise services                | Yes                | No          |
| `ApplicationEvent` publication     | Yes                | No          |
| `Environment` abstraction          | Yes                | No          |

The hierarchy of `ApplicationContext` classes looks like:
```
ApplicationContext
|
+-- AbstractApplicationContext : can detect special beans like post processors
    |
    +-- GenericApplicationContext : contains DefaultListableBeanFactory
        |
        +-- AnnotationConfigApplicationContext
```

Like `AnnotationConfigApplicationContext`, there are several concrete implementations catering to different possible sources of configuration metadata. For example: `ClassPathXmlApplicationContext`, `FileSystemXmlApplicationContext`, `GenericGroovyApplicationContext`, etc. These `ApplicationContext` delegate the resposibility of registering and creating bean instances to the `DefaultListableBeanFactory` contained within its parent class.

Configuration metadata can be supplied as XML, annotations (ver 2.5) or as Jave code (ver 3.0). Once the configuration is ready, we can get the bean instance by:

In [ ]:
// Create the context
ApplicationContext context = new ClassPathXmlApplicationContext("services.xml");

// Get the bean
UserService userService = context.getBean("userService", UserService.class);

### AnnotationConfigApplicationContext
Reads classes annotated with `@Component` to obtain bean definition and creates bean instances from it. We can also make use of `@Configuration` since it is also a component. Another related annotation is `@ComponentScan` which specifies the packages to search for components.  `@Configuration` is defined as:

In [ ]:
@Target({ElementType.TYPE})
@Retention(RetentionPolicy.RUNTIME)
@Documented
@Component
public @interface Configuration {
    @AliasFor(annotation = Component.class)
    String value() default "";

    boolean proxyBeanMethods() default true;
}

And `@ComponentScan` as below. By default, `@ComponentScan` would scan for components in the same package (and its subpackages).

In [ ]:
public @interface ComponentScan {
    // Array of package names to scan
    String[] basePackages() default {};

    // Type safe alternative to above
    Class<?>[] basePackageClasses() default {};
    
    // Lazy init the discovered components?
    boolean lazyInit() default false;
    
    // Specifies which types are eligible for component scanning
    Filter[] includeFilters() default {};

    // Specifies which types are not eligible for component scanning
    Filter[] excludeFilters() default {};
    
    @interface Filter {
        // The type of filter to use
        FilterType type() default FilterType.ANNOTATION;
        
        // The class or classes to use as the filter
        Class<?>[] classes() default {};
        
        // The pattern (or patterns) to use for the filter, as an alternative to specifying a Class value
        String[] pattern() default {};
        
        // Other fields
    }
    
    // Other fields
}

There are quite a few different ways to filter out components using the `@Filter` annotation. For example, to filter out using regular expression:

In [ ]:
@Configuration
// Will include compenents named Compiler, Processor, etc. But not Store, Repository
@ComponentScan(includeFilters = @Filter(type = FilterType.REGEX, pattern = ".*[er]"))
public class ComponentScanRegexFilterApp { }

Another way is to include/exclude classes having certain annotations:

In [ ]:
@Configuration
// Will pick up components annotated with @Animal
@ComponentScan(includeFilters = @Filter(type = FilterType.ANNOTATION, classes = Animal.class))
public class ComponentScanAnnotationFilterApp { }

Some other type of `FilterType`s are:
- CUSTOM
- ASPECTJ
- ASSIGNABLE_TYPE

## Bean Definition
Some important bean definition properties:

### Bean Identifier
We can supply an *id* or *name* to a bean. However this is not mandatory and Spring container would automatically generate a unique name for that bean.

In [ ]:
@Component
// Spring names this bean as cat
public class Cat extends Animal{
}

@Component("myDog")
// We provide name explicitly
public class Dog extends Animal{
}

@Component
@Qualifier("petGoat")
// Another way to name our bean
public class Goat extends Animal{
}

When using `@Bean` Spring uses method name as bean name

In [ ]:
@Bean
// Bean named as getCat
public Cat getCat(){
    return new Cat();
}

@Bean("petDog")
// Bean named as petDog
public Dog getDog(){
    return new Dog();
}

### Dependencies
Dependencies can be injected using either constructors or setters. Constructor-based DI is accomplished by the container invoking a constructor with a number of arguments, each representing a dependency. Setter-based DI is accomplished by the container calling setter methods on beans after invoking a no-argument constructor or a no-argument static factory method to instantiate bean.

**Lazy initialization:** by default Spring would eagerly create and configure all singleton beans as part of the initialization process. `@Lazy` annotation can be used to specify that a particular bean should not be eagerly initialized. If applied to: 
- `@Component` `@Bean` these beans will be lazily initialized
- `@Configuration` all beans will be lazily initialized

**Autowiring:** Use `@Autowired` annotation to specify that a dependency should be automatically injected. `@Autowired` can be applied to:
- field : works even for private member
- method
- constructor
- method parameter : not really used much  

By default Spring autowires by type, to autowire by name:

In [ ]:
@Autowired
@Qualifier("ldap")
private NamingService nameService;

Other possibility is that we can mark one of the bean as primary candidate for autowiring

In [ ]:
@Component
@Primary
public class LdapNamingService{
    // ...
}

Spring by default autowires constructor arguments, we don’t have to use the `@Autowired` annotation. Instead of `@Autowired` we can also use Java EE `@Inject` annotation. And we can also replace `@Autowired` and `@Qualifier` combination with `@Resource`.

We can also autowire into a list:

In [ ]:
@Component
public class Car implements Vehicle {
}

@Component
public class Bus implements Vehicle {
}

@Component
public class User {
   @Autowired
   List<Vehicle> vehicles; // contains both car and bus
}

### Bean Scopes
The Spring Framework supports six scopes, four of which are available only if we use a web-aware `ApplicationContext`.
| Scope       | Description                                                                                                                                                                            |
|-------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| singleton   | (Default) Scopes a single bean definition to a single object instance for each Spring IoC container.                                                                                   |
| prototype   | Scopes a single bean definition to any number of object instances.                                                                                                                     |
| request     | Scopes a single bean definition to the lifecycle of a single HTTP request. That is, each HTTP request has its own instance of a bean created off the back of a single bean definition. |
| session     | Scopes a single bean definition to the lifecycle of an HTTP Session.                                                                                                                   |
| application | Scopes a single bean definition to the lifecycle of a ServletContext.                                                                                                                  |
| websocket   | Scopes a single bean definition to the lifecycle of a WebSocket.                                                                                                                       |

Use `@Scope`, `@RequestScope`, `@SessionScope`, `@ApplicationScope` annotations to mark a bean’s scope as required.

## Bean Lifecycle
<img src="./images/bean_lifecycle.png" />  

1. Read Bean definitions from various sources and form `BeanDefinition` objects and register with `BeanDefinitionRegistry`  
2. **BeanFactoryPostProcessor:** Factory hook that allows for custom modification of an application context's bean definitions. It is executed before instantiation of bean. The interface looks like:

In [ ]:
@FunctionalInterface
public interface BeanFactoryPostProcessor {
    void postProcessBeanFactory(ConfigurableListableBeanFactory beanFactory) throws BeansException;
}

Several implementations of `BeanFactoryPostProcessor` is provided by Spring as listed below:  
**PropertySourcesPlaceholderConfigurer:** resolves ${...} placeholders within bean definition property values and `@Value` annotations against the current Spring `Environment` and its set of `PropertySources`:

In [ ]:
@Configuration
@PropertySource("classpath:foo.properties") // used in conjunction with @Configuration
public class PropertiesWithJavaConfig {
    
    @Value( "${jdbc.url}" ) // from foo.properties
    private String jdbcUrl;
    
    //...
}

// Property source can also have the following variants:
@Configuration
@PropertySource({"classpath:foo.properties", "classpath:bar.properties"})
class AnotherConfiguration {
    // ...
}

@Configuration
@PropertySources({
    @PropertySource("classpath:foo.properties"),
    @PropertySource("classpath:bar.properties")
})
class YetAnotherConfiguration {
    // ...
}

An `ApplicationContext` auto-detects `BeanFactoryPostProcessor` beans in its bean definitions and applies them before any other beans get created. A `BeanFactoryPostProcessor` may also be registered programmatically with a `ConfigurableApplicationContext`. 

`BeanFactoryPostProcessor` beans that are autodetected in an `ApplicationContext` will be ordered according to `PriorityOrdered` and `Ordered` semantics. In contrast, `BeanFactoryPostProcessor` beans that are registered programmatically with a `ConfigurableApplicationContext` will be applied in the order of registration. Furthermore, the `@Order` annotation is not taken into account for `BeanFactoryPostProcessor` beans.

3. **Bean Instantiation:** Bean is constructed and its dependencies injected. All `*Aware` beans are also injected in this phase. Some examples of `*Aware` beans are:  
    a. `ApplicationContextAware` interface to get access to the IOC container inside a bean  
    b. `BeanNameAware` interface lets a bean access its name provided by the container  
    c. `EnvironmentAware`, etc  
    We don’t even need to use the `Aware` interfaces, Spring can automatically inject `ApplicationContext`, `Environment`, etc. Many other objects can also get autowired without using equivalent Aware interface such as `BeanFactory`, `MessageSource`, `Environment`, `ResourceLoader`, `ApplicationEventPublisher`, etc
4. **BeanPostProcessor:** Factory hook that allows for custom modification of new bean instances — for example, checking for marker interfaces or wrapping beans with proxies. Provides two methods - one that acts before initialization and the other after initialization as shown below:

In [ ]:
public interface BeanPostProcessor {
    @Nullable
    default Object postProcessBeforeInitialization(Object bean, String beanName) throws BeansException {
        return bean;
    }

    @Nullable
    default Object postProcessAfterInitialization(Object bean, String beanName) throws BeansException {
        return bean;
    }
}

It follows the same ordering logic that `BeanFactoryPostProcessor`s follow.  

5. **Initialization:** Spring supports multiple initialization methods in the order as follows:  
    a. Methods annotated with `@PostConstruct` (required JavaEE)  
    b. `afterPropertiesSet()` as defined by the `InitializingBean` callback interface  
    c. A custom configured `init()` method by using `@Bean(name = "demoBean", initMethod = "init")`  

## Environment
Spring `Environment` is combination of `Profile` and `PropertySource`. It is a convenient abstraction to access key value pairs present in OS environment variables, JVM system properties, JNDI, servlet context parameters, ad-hoc `Properties` objects, `Map` objects, etc.

In [ ]:
AnnotationConfigApplicationContext annotationConfigApplicationContext = 
    new AnnotationConfigApplicationContext();
Environment environment = annotationConfigApplicationContext.getEnvironment();

This returns instance of `StandardEnvironment` which has two property sources
- `System.getProperties()` (JVM)
- `System.env()` (environment variables)

Other application context may supply different implementation of environment with different `PropertySource`s. For example, web application context returns `StandardServletEnvironment` with the following priority:
- `ServletConfig` parameters (if applicable — for eg, in case of a `DispatcherServlet` context)
- `ServletContext` parameters (web.xml context-param entries)
- JNDI environment variables (java:comp/env/ entries)
- JVM system properties (-D command-line arguments)
- JVM system environment (operating system environment variables)

We can configure an `Environment` and add `PropertySource`:

In [ ]:
ConfigurableApplicationContext configurableApplicationContext = 
    new GenericApplicationContext();
ConfigurableEnvironment configurableEnvironment = configurableApplicationContext.getEnvironment();

// Configurable environment is implemented by StandardEnvironment
MutablePropertySources sources = configurableEnvironment.getPropertySources();
sources.addFirst( // addFirst gives this PropertySource highest precedence
    new ResourcePropertySource(new ClassPathResource("container/values.properties")));

for (PropertySource<?> propertySource : sources) {
    System.out.println("Name =  " + propertySource.getName() +
            "\nSource = " + propertySource.getSource().getClass() + "\n");
}

There are multiple different flavours of `PropertySource` as listed in the hierarchy below:  
```
PropertySource (abstract)
    |
    +-- EnumerablePropertySource (abstract)
            |
            +-- CommandLinePropertySource (abstract)
            |       |
            |       +-- SimpleCommandLinePropertySource
            +-- MapPropertySource (sources properties from Map) 
                    |
                    +-- PropertiesPropertySource (sources properties from Properties)
                            |
                            +-- ResourcePropertySource (sources from Resource)
```

**CommandLinePropertySource:** sources its properties from command line arguments. For example, if we pass the `--o1=v1 --o2` as the commandline argument, following holds true:

In [ ]:
// use of -- incdicates option argument
CommandLinePropertySource<?> ps = ...
assert ps.containsProperty("o1") == true;
assert ps.containsProperty("o2") == true;
assert ps.containsProperty("o3") == false;
assert ps.getProperty("o1").equals("v1");
assert ps.getProperty("o2").equals(""); // and not null
assert ps.getProperty("o3") == null;

Any arguments supplied without an option-style prefix such as "-" or "--" are considered "non-option arguments" and available through the special "nonOptionArgs" property. For example, consider `--o1=v1 --o2=v2 /path/to/file1 /path/to/file2` being passed as argument, then:

In [ ]:
CommandLinePropertySource<?> ps = ...
assert ps.containsProperty("o1") == true;
assert ps.containsProperty("o2") == true;
assert ps.containsProperty("nonOptionArgs") == true;
assert ps.getProperty("o1").equals("v1");
assert ps.getProperty("o2").equals("v2");
assert ps.getProperty("nonOptionArgs")
    .equals("/path/to/file1,/path/to/file2"); // all nonoption arguments
                                              // get concatenated

// When used in conjunction with the Spring Environment abstraction, 
// this comma-delimited string may easily be converted to a String array or list:
Environment env = applicationContext.getEnvironment();
String[] nonOptionArgs = env.getProperty("nonOptionArgs", String[].class);
assert nonOptionArgs[0].equals("/path/to/file1");
assert nonOptionArgs[1].equals("/path/to/file2");

**Annotation:** `@PropertySource` annotation provides a convenient and declarative mechanism for adding a `PropertySource` to Spring’s `Environment`.

In [ ]:
@Configuration
@PropertySource("classpath:/com/example/app.properties") // can be specified multiple times
public class AppConfig {

`PropertySource` location can also contain `${}` placeholders. These will be resolved against pre-existing property sources.

### Profile
The `@Profile` annotation lets us indicate that a component is eligible for registration when one or more specified profiles are active.

In [ ]:
@Configuration
public class AppConfig {

    @Bean("dataSource")
    @Profile("development") 
    public DataSource standaloneDataSource() {
        return new EmbeddedDatabaseBuilder()
            .setType(EmbeddedDatabaseType.HSQL)
            .addScript("classpath:com/bank/config/sql/schema.sql")
            .addScript("classpath:com/bank/config/sql/test-data.sql")
            .build();
    }

    @Bean("dataSource")
    @Profile("production") 
    public DataSource jndiDataSource() throws Exception {
        Context ctx = new InitialContext();
        return (DataSource) ctx.lookup("java:comp/env/jdbc/datasource");
    }
}

`@Profile` annotation can also be applied at class level. Profile string may contain a simple profile name (for example, production) or a profile expression. A profile expression allows for more complicated profile logic to be expressed (for example, production & us-east). The following operators are supported in profile expressions:
- !: A logical “not” of the profile
- &: A logical “and” of the profiles
- |: A logical “or” of the profiles

There can be more than one profile active at a time. To see which profiles are currently active:

In [ ]:
String[] activeProfiles = environment.getActiveProfiles();
System.out.println(Arrays.toString(activeProfiles));

**Activating a profile programmatically:**

In [ ]:
AnnotationConfigApplicationContext ctx = new AnnotationConfigApplicationContext(AppConfig.class);
ctx.getEnvironment().setActiveProfiles("development");
ctx.refresh();

**Activating a profile declaratively:** set `spring.profiles.active` to comma separated list of profile names. We can pass this as our program argument.  

The below code will throw `NoUniqueBeanDefinitionException`. This is because the beans with no profiles attached to them are always loaded.

In [ ]:
@Configuration
public class ProfilesDemo {
    @Component
    class DateFormatter {
        @Autowired
        private String format;

        public String format(Date date){
            return new SimpleDateFormat(format).format(date);
        }
    }

    @Bean
    @Profile("us")
    public String getUSDateFormat(){
        return "MM/dd/yyyy";
    }

    @Bean
    @Profile("utc")
    public String getUTCDateFormat(){
        return "dd-MM-yyyy";
    }

    @Bean
    public String getDefaultDateFormat(){
        return "dd/MM/yyyy";
    }

    public static void main(String[] args) {
        AnnotationConfigApplicationContext context =  new AnnotationConfigApplicationContext();
        context.register(ProfilesDemo.class);
        context.getEnvironment().getSystemProperties()
            .put(AbstractEnvironment.ACTIVE_PROFILES_PROPERTY_NAME, "us");
        context.refresh();

        System.out.println(Arrays.toString(context.getEnvironment().getActiveProfiles()));

        DateFormatter formatter = context.getBean(DateFormatter.class);
        System.out.println(formatter.format(new Date()));
    }
}

We can have a bean being created when no profile is active and another bean of same type created when a profile is active by using default profile:

In [ ]:
// created when us profile is active
@Bean
@Profile("us")
public String getUSDateFormat(){
    return "MM/dd/yyyy";
}

// created when utc profile is active
@Bean
@Profile("utc")
public String getUTCDateFormat(){
    return "dd-MM-yyyy";
}

// created when no profile is active
@Bean
@Profile("default")
public String getDefaultDateFormat(){
    return "dd/MM/yyyy";
}

## Testing
Spring provides a way to create application context for tests and caches it such that the context doesn't have to reload for each unit test. The `@ContextConfiguration` annotation lets us specify the classes (`@Configuration` or `@Component`) that configuration can be loaded from:

In [ ]:
@ContextConfiguration(classes = {AppConfig.class})
@RunWith(SpringRunner.class) // Need to use this runner to execute tests
public class BaseTest {
    @Autowired
    private ApplicationContext context;

    @Test
    public void test() {
        Assert.assertEquals("Scheme doesn't match", "http", context.getBean(URI.class).getScheme());
    }
}

`@ContextConfiguration` also supports specifying `ApplicationContextInitializer`. This class lets us specify code that gets executed before the Spring application context gets completely created. Example usage:

In [ ]:
public class CustomApplicationContextInitializer implements ApplicationContextInitializer<ConfigurableApplicationContext> {
     @Override
     public void initialize(ConfigurableApplicationContext ac) {
         ConfigurableEnvironment environment = ac.getEnvironment();
         environment.addActiveProfile("test");
     }
}

We can specify list of application context initializers using the `initializers` parameter: `@ContextConfiguration(initializers = {CustomApplicationContextInitializer.class})`. 

`@ContextConfiguration` supports boolean `inheritLocations` and `inheritInitializers` attributes that denote whether resource locations or component classes and context initializers declared by superclasses should be inherited. The default value for both flags is true. This means that a test class inherits the resource locations or component classes as well as the context initializers declared by any superclasses.

To change application context type to `WebApplicationContext`, use `@WebAppConfiguration` annotation.

### `@DirtiesContext`
This annotation indicates that the application context may have been modified, therefore the application context cache needs to be discarded and application context needs to be reloaded. Example without using `@DirtiesContext`:

In [ ]:
@ContextConfiguration
@RunWith(SpringRunner.class)
public class DirtiesContextTest {
    
    // In case we don't specify classes in @ContextConfiguration, Spring looks for
    // static inner @Configuration classes
    @Configuration
    static class TestConfig {
        @Bean
        public UserCache cache() {
            return new UserCache();
        }
    }

    @Autowired
    private UserCache cache;

    @Test
    public void testOne() {
        cache.addUser(new User("root", "root"));
        Assert.assertEquals("[User[username=root, password=root]]", cache.toString());
    }

    // This test fails because previous test dirtied the application context
    // by adding a User in the UserCache
    @Test
    public void testTwo() {
        cache.addUser(new User("admin", "admin"));
        Assert.assertEquals("[User[username=admin, password=admin]]", cache.toString());
    }
}

record User(String username, String password) {};

class UserCache {
    private final Set<User> users = new HashSet<>();

    public void addUser(User user) {
        users.add(user);
    }

    @Override
    public String toString() {
        return users.toString();
    }
}

`@DirtiesContext` can be applied to individual test methods or at the entire class level. Both the variants support the following modes:  
**MethodMode:**
- `BEFORE_CLASS`: application context is marked as dirty before corresponding test method
- `AFTER_CLASS`: application context is marked as dirty after corresponding test method

Similar option exists for **ClassMode**:
- `BEFORE_CLASS`: application context is marked as dirty before test class
- `BEFORE_EACH_TEST_METHOD`: application context is marked as dirty before each method of test class
- `AFTER_EACH_TEST_METHOD`: application context is marked as dirty before each method of test class
- `AFTER_CLASS`: application context is marked as dirty after test class

Therefore in the previous example, we just have to do the following modification:

In [ ]:
@ContextConfiguration
@RunWith(SpringRunner.class)
@DirtiesContext(classMode = DirtiesContext.ClassMode.AFTER_EACH_TEST_METHOD)
public class DirtiesContextTest {

### `@TestPropertySource`
Used to specify a property resource (for example a .properties file) or list of key-value pairs to source properties from for the test:

In [ ]:
@ContextConfiguration
@TestPropertySource(
    locations = {"classpath:/com/example/test.properties"},
    properties = {"timezone = GMT", "port: 4242"} // both key = value and key : value are accepted
) 
class MyIntegrationTests {